In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
df_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
df_train['seq'] = df_train.sort_values(['breath_id', 'time_step']).groupby('breath_id')['id'].cumcount()
df_test['seq'] = df_test.sort_values(['breath_id', 'time_step']).groupby('breath_id')['id'].cumcount()
df_train.drop(['id', 'breath_id'], axis = 1, inplace = True)
df_test.drop(['breath_id'], axis = 1, inplace = True)
test_id = df_test.pop('id')

In [ ]:
HOURS = 2

predictor = TabularPredictor(label='pressure', eval_metric='mean_absolute_error')
predictor.fit(train_data= df_train,
              time_limit= int(3600*HOURS),
              ag_args_fit={'num_gpus': 1},
              presets='best_quality',
              excluded_model_types=['KNN'],
              verbosity=2)

In [ ]:
pred_probas = predictor.predict_proba(df_test)

In [ ]:
df_test['id'] = test_id
df_test['pressure'] = pred_probas
df_sub = df_test.loc[:, ['id', 'pressure']]
df_sub.to_csv('submission.csv', index = False)